# Experiment pipeline

1. load data: in Uncertainty class __init__. Instantiate an Uncertainty instance.
    - load trained model
    - for the trained model, and its validation set (w/ corresponding activations, predictions, and gt), instantiate a CulpritNeuronScore instance, for further culprit score compute
    - load query dataset activations, predictions, gt. (gt is only used for validate the methods, in query time with user, gt is unknown)


2. compute culprit matrix for a given trained model: get_culprit_matrix(), get_baseline() for random culprit comparison as baseline.
    - specify: model, culprit method
    
    
3. compute uncertain matrix given the culprit mtx, and the query data activations: get_uncertain_matrix()


4. visualize the results, compute the correlation between gt error and the uncertain mtx.

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from culprit import *
from uncertainty import *
import json
import datetime
import csv


%load_ext autoreload
%autoreload 2

In [16]:
os.environ["CUDA_VISIBLE_DEVICES"]= '1'
models = ['skinmodel/checkpoint.pth']
experiment_saved_path = './saved' # it has the experiment activation maps, and saved exp data as a folder
clpt_methods = ['select']
sim_methods = ['cosine']#, 'mi']
gt_error_methods = ['l1']#, 'ce']
model_path = 'skinmodel/checkpoint.pth'

In [27]:
model_name = models[0].strip('/')#[-1]#.strip('.')[0]
model_name

'skinmodel/checkpoint.pth'

In [17]:
def pipeline(clpt_method, sim_method, gt_error_method, model, \
             experiment_saved_path = './saved', \
             query = 'config_skin_alexnet_query.json', val = 'config_skin_alexnet_val.json', \
             baseline = True):
    
    # 1. load data: in Uncertainty class init. Instantiate an Uncertainty instance
    uncty = Uncertainty(experiment_saved_path, model_path)
    gt_error_methods = {'l1': uncty.get_generalize_error, 'ce': uncty.get_generalize_error_ce }
    gt, pred = uncty.get_query_gt_pred()
    
    # 2. compute culprit matrix for a given trained model, get_baseline() for random culprit comparison as baseline.
    clpt_mtx = uncty.get_culprit_matrix(clpt_method)
    # 3. compute uncertain matrix given the culprit mtx, and the query data activations
    uncty_mtx = uncty.get_uncertain_matrix(clpt_mtx, uncty.query_actv, sim_method)
    
    # 2.1 baseline
    bl_clpt_mtx = uncty.get_baseline(clpt_mtx)
    bl_uncty_mtx = uncty.get_uncertain_matrix(bl_clpt_mtx, uncty.query_actv, sim_method)
    
    # 4. prepare for vis
    actv_map_shape = uncty.get_actv_shape()
    
    # save the data as csv for vis
    
    # visualize the save experiment results
    timestamp = datetime.datetime.now().strftime('%m%d_%H%M%S')
    model_name = model.stripe('/')[-1].stripe('.')[0]
    experiment_saved_subfolder = timestamp + '_' + model_name + '_' + clpt_method + '_' + sim_method + '_' + gt_error_method
#         uncty.experiment_results_visualize(gt_error_method, experiment_saved_path, experiment_saved_subfolder)
    # interactive vis: select target layer, show correlation with gt

    error = gt_error_methods[gt_error_method](gt, pred)
    # create subfolder for results saving
    subdir = experiment_saved_path + '/' + experiment_saved_subfolder
    if not os.path.exists(subdir):
        os.mkdir(subdir)
    # save bl_uncty_mtx
    with open(subdir+"/"+"bl_uncty_mtx.csv","w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(bl_uncty_mtx)
    with open(subdir+"/"+"uncty_mtx.csv","w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(uncty_mtx)    
    with open(subdir+"/"+"gt_error.csv","w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(error)  
#         plt.scatter(error.reshape(-1), uncty_mtx.reshape(-1))
#         # vis1. correlation scatter plot
#         plt.plot(error.reshape(-1))
#         plt.show()
#         plt.clf() 
#         plt.close()
#         # vis2. signal plot over datapoints and their classes                 
#         plt.plot(uncty_mtx.reshape(-1))
#         plt.legend(['gt error', 'proposed_uncertainty'])
#         plt.title('the gt error and the proposed uncertainty for the query 150 images')
#         plt.show()
#         plt.clf() 
#         plt.close()
    return uncty

In [18]:
uncty = None
for model in models:
    for clpt_method in clpt_methods:
        for sim_method in sim_methods:
            for gt_error_method in gt_error_methods:
                uncty = pipeline(clpt_method, sim_method, gt_error_method, model)

*** actv shape (ignore dim 0 - batch size) is: [torch.Size([64, 64, 55, 55]), torch.Size([64, 192, 27, 27]), torch.Size([64, 384, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 4096]), torch.Size([64, 4096]), torch.Size([64, 2])] .
*** data loaded ***
*** Flattened actv vector shape is torch.Size([150, 9346]).
*** actv shape (ignore dim 0 - batch size) is: [torch.Size([64, 64, 55, 55]), torch.Size([64, 192, 27, 27]), torch.Size([64, 384, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 4096]), torch.Size([64, 4096]), torch.Size([64, 2])] .
*** data loaded ***
*** label size is torch.Size([600]), right prediction is 459.
*** feature shape is torch.Size([600, 9346]).
*** right_actv shape is torch.Size([459, 9346]), wrong_actv shape is torch.Size([141, 9346]).
*** select method, culprit matrix shape is: (2, 9346)


/local-scratch/weinaj/pytorch_classification_template/culprit.py:149: RuntimeWarning:

divide by zero encountered in true_divide

/local-scratch/weinaj/pytorch_classification_template/culprit.py:149: RuntimeWarning:

invalid value encountered in true_divide



FileNotFoundError: [Errno 2] No such file or directory: './saved/0321_125624_skinmodel/checkpoint.pth_select_cosine_l1'

# use the saved data for results visualization

In [15]:
import plotly.plotly as py
import plotly.graph_objs as go